<a href="https://colab.research.google.com/github/pnidatunca/Smart-Vehicle-License-Plate-Recognition-System/blob/main/VLM_finalSon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Kütüphaneleri Yüklüyoruz
!pip install ultralytics
!pip install easyocr
!pip install transformers
!pip install torch torchvision
!pip install opencv-python-headless
print("Kütüphaneler başarıyla yüklendi.")

In [ ]:
# Import
import cv2
import easyocr
import numpy as np
from google.colab.patches import cv2_imshow
from ultralytics import YOLO
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
import re  # Regex kütüphanesi (Plaka formatı kontrolü için)

print("Kütüphaneler yüklendi.")

In [ ]:
# Standart Modelleri Yükle

print("Standart modeller yükleniyor.")

vehicle_model = YOLO('yolov8n.pt')

# OCR Motoru ve BLIP modelleri EnhancedPlateReader içinde yüklenecek.
# Bu değişkenler global olarak tanımlanmayacak.

In [ ]:
# Kontrast Güçlendirici (CLAHE) ve Karakter Tamiri Yapıyoruz

import easyocr
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import cv2
import numpy as np
import re
from PIL import Image

class EnhancedPlateReader:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Gelişmiş Okuyucu Yükleniyor... (Cihaz: {self.device})")

        self.detector = easyocr.Reader(['en'], gpu=(self.device == "cuda"))
        self.blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        self.blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(self.device)

        # Karakter düzeltme haritası (Harf -> Rakam)  (Yanlış Okumayı azaltmak için)
        self.char_fix_map = {
            'O': '0', 'D': '0', 'Q': '0', 'U': '0',
            'I': '1', 'L': '1',
            'Z': '2',
            'B': '8',
            'S': '5',
            'G': '6'
        }

    def describe_vehicle(self, img_crop):
        try:
            pil_img = Image.fromarray(cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB))
            inputs = self.blip_processor(pil_img, return_tensors="pt").to(self.device)
            out = self.blip_model.generate(**inputs)
            return self.blip_processor.decode(out[0], skip_special_tokens=True)
        except:
            return ""

    def preprocess_image(self, img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Bu işlem gölgede kalan veya düşük kontrastlı yazıları belirginleştirir.
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        enhanced = clahe.apply(gray)

        denoised = cv2.fastNlMeansDenoising(enhanced, None, 10, 7, 21)

        return denoised

    def smart_fix_text(self, text):
        """
        'KA0O4' gibi hatalı okumaları 'KA004' yapar.
         Regex'e uymayan karakterleri zorla düzeltir.
        """
        text_list = list(text)

        # Kabaca Türk plaka yapısı: RAKAM - HARF - RAKAM
        # Bu yüzden baştaki ve sondaki harfleri rakama zorlayabiliriz.
        # Son 2-3 karakter genelde rakamdır. Eğer orada harf varsa (O, I, Z) değiştir.
        for i in range(len(text_list) - 4, len(text_list)):
            if i >= 0:
                char = text_list[i]
                if char in self.char_fix_map:
                    text_list[i] = self.char_fix_map[char]

        # İlk 2 karakter genelde rakamdır .
        for i in range(2):
            if i < len(text_list):
                char = text_list[i]
                if char in self.char_fix_map:
                    text_list[i] = self.char_fix_map[char]

        return "".join(text_list)

    def validate_format(self, text):
        clean = ''.join(e for e in text if e.isalnum()).upper()

        # Önce ilk halini deniyoruz
        pattern = re.compile(r"^(\d{2})([A-Z]{1,3})(\d{2,5})$")
        if pattern.match(clean):
            return True, clean

        # Bulunamadıysa TAMİR et ve tekrar dene
        fixed_text = self.smart_fix_text(clean)
        if pattern.match(fixed_text):
            print(f"-> Tamir Edildi: {clean} => {fixed_text}")
            return True, fixed_text

        return False, None

    def cluster_into_lines(self, detections, y_tolerance=20):
        if not detections: return []

        blocks = []
        for (box, text, prob) in detections:
            (tl, tr, br, bl) = box
            y_center = (tl[1] + bl[1]) / 2

            clean_text = ''.join(e for e in text if e.isalnum()).upper()
            if len(clean_text) > 0 and prob > 0.1:
                blocks.append({'y': y_center, 'x': tl[0], 'text': clean_text})

        blocks.sort(key=lambda b: b['y'])

        lines = []
        if not blocks: return []

        current_line = [blocks[0]]
        for i in range(1, len(blocks)):
            block = blocks[i]
            prev_block = current_line[-1]

            if abs(block['y'] - prev_block['y']) < y_tolerance:
                current_line.append(block)
            else:
                lines.append(current_line)
                current_line = [block]
        lines.append(current_line)
        return lines

    def read_plate(self, original_img, bbox):
        x1, y1, x2, y2 = map(int, bbox)
        vehicle_crop = original_img[y1:y2, x1:x2]
        h, w = vehicle_crop.shape[:2]

        roi_img = vehicle_crop[int(h*0.35):h, 0:w]
        # 1. Görüntüyü İyileştir (CLAHE)
        processed_img = self.preprocess_image(roi_img)

        # 2. EasyOCR Tara
        detections = self.detector.readtext(processed_img)

        # 3. Satırlara Böl
        lines = self.cluster_into_lines(detections)

        print(f"\n--- Gelişmiş Analiz ({len(lines)} Satır) ---")

        for i, line in enumerate(lines):
            line.sort(key=lambda b: b['x'])
            line_text = "".join([b['text'] for b in line])

            print(f"Satır {i+1} (Ham): {line_text}")

            # Format Kontrolü ve Tamir
            is_valid, valid_text = self.validate_format(line_text)

            if is_valid:
                return valid_text

            # Regex ile ayıklama (Dirty string içinden)
            # Örn: TR14KA004 -> 14KA004
            search_pattern = r"(\d{2}[A-Z]{1,3}\d{2,5})"

            # Önce tamir et sonra ara
            fixed_line = self.smart_fix_text(line_text)
            match = re.search(search_pattern, fixed_line)
            if match:
                print(f"-> Regex ile kurtarıldı: {match.group(1)}")
                return match.group(1)

        return "---"

print("Gelişmiş Okuyucu (CLAHE + SmartFix) Hazır.")
reader = EnhancedPlateReader()

In [ ]:
# Veritabanı için json doyası oluşturuyoruz
import json

# Veritabanımız
veritabani = [
    "34ABC123",
    "66AAP914",
    "03JAA161",
    "35BZA442",
    "06YHB12",
    "20ACE855",
    "35EE510"
]

# Dosyayı kaydet
with open('abone_listesi.json', 'w') as f:
    json.dump(veritabani, f)

print("'abone_listesi.json' dosyası oluşturuldu.")

In [ ]:
# Gerekli kütüphane (Resimleri ekranda görmek için)
from google.colab.patches import cv2_imshow
import cv2
import os

def process_vehicle_entry(image_path):
    # Resmi okuyoruz
    img = cv2.imread(image_path)
    if img is None:
        print(f"Dosya açılamadı: {image_path}")
        return

    # Abone listesini her seferinde güncelliyoruz
    AUTHORIZED_PLATES = load_authorized_plates()

    # YOLO modelini çalıştır (Sadece Araba, Otobüs ve Tırları bulsun diye filtreledim)
    results = vehicle_model(img, classes=[2, 5, 7], conf=0.45, verbose=False)

    found_any = False
    system_logs = []

    for r in results:
        for box in r.boxes:
            found_any = True

            # Koordinatları al
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls_id = int(box.cls[0])
            label = vehicle_model.names[cls_id].upper()

            # Aracın olduğu kareyi kesip alıyoruz (Plaka okuma için)
            vehicle_crop = img[y1:y2, x1:x2]

            # 1. Adım: Plakayı okutuyoruz
            plate_text = reader.read_plate(img, [x1, y1, x2, y2])

            # 2. Adım: Aracı tarif et (Rengi vs. neymiş)
            description = reader.describe_vehicle(vehicle_crop)

            # 3. Adım: Giriş izni kontrolü (Kurallarımız burada)
            decision = ""
            color = (0, 0, 0) # Varsayılan siyah

            # Öncelik 1: Abone ise direkt geçsin
            if plate_text in AUTHORIZED_PLATES:
                decision = "ONAY"
                color = (0, 255, 0) # Yeşil

            # Öncelik 2: Yasaklı araç tipleri (Kamyon/Otobüs)
            elif "TRUCK" in label:
                decision = "RED (KAMYON)"
                color = (0, 0, 255) # Kırmızı
            elif "BUS" in label:
                decision = "RED (OTOBUS)"
                color = (0, 0, 255) # Kırmızı

            # Öncelik 3: Diğer durumlar (Misafir veya Okunamadı)
            else:
                if plate_text == "---":
                    decision = "BELIRSIZ"
                    color = (0, 255, 255) # Sarı
                else:
                    decision = "RED (KAYITSIZ)"
                    color = (0, 0, 255) # Kırmızı

            # Kayıt listesine ekle
            system_logs.append({
                "Arac": label,
                "Plaka": plate_text,
                "Kod": decision,
                "Yorum": description
            })

            # --- Çizim İşlemleri ---
            # Aracın etrafına kutu çiz
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 4)

            # Üstüne etiketi yaz
            header = f"{label} | {decision}"
            (w, h), _ = cv2.getTextSize(header, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)

            cv2.rectangle(img, (x1, y1 - 35), (x1 + w, y1), color, -1)
            cv2.putText(img, header, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

            # Eğer plaka okunduysa altına yaz
            if plate_text != "---":
                cv2.putText(img, plate_text, (x1, y2+30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Sonuçları Ekrana Bas (Resim varsa göster)
    if found_any:
        print(f"\n📸 İşlenen: {os.path.basename(image_path)}")

        # Colab'da resmi gösteren komut
        cv2_imshow(img)

        print("-" * 60)
        for log in system_logs:
            emoji = "✅" if log["Kod"] == "ONAY" else "⛔" if "RED" in log["Kod"] else "⚠️"
            print(f"{emoji} {log['Arac']} -> {log['Plaka']} ({log['Kod']})")
        print("-" * 60 + "\n")
    else:
        print(f"Araç bulunamadı: {os.path.basename(image_path)}")

In [ ]:
import glob

# Test edilecek klasör yolu
KLASOR_YOLU = "/content/test_Photos"

print(f"'{KLASOR_YOLU}' klasörü taranıyor...\n")

# Klasördeki tüm resimleri bul
resimler = sorted([f for f in glob.glob(os.path.join(KLASOR_YOLU, "*.*"))
                   if f.lower().endswith(('.jpg', '.jpeg', '.png', '.webp'))])

if not resimler:
    print("Klasörde resim bulunamadı! Lütfen resim yüklediğinizden emin olun.")
else:
    # Her bir resim için yazdığınız fonksiyonu çağır
    for resim_yolu in resimler:
        process_vehicle_entry(resim_yolu)